# Sentinel-1 SAR Data Retrieval with Sentinel Hub

This notebook demonstrates how to retrieve Sentinel-1 SAR (Synthetic Aperture Radar) data using the Sentinel Hub API. The data is downloaded based on a specified area of interest (AOI) defined in a GeoJSON file.

## Step 0: Install dependencies


In [353]:
%pip install geopandas sentinelhub python-dotenv matplotlib numpy

Note: you may need to restart the kernel to use updated packages.


## Step 1: Import Necessary Libraries

In [354]:
# Import necessary libraries

import geopandas as gpd
from sentinelhub import (
    SentinelHubDownloadClient,
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
)
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import numpy as np

## Step 2: Load Environment Variables

In [355]:
load_dotenv()

True

## Step 3: Load Sentinel Hub Credentials

In this step, we configure the Sentinel Hub API credentials needed to authenticate requests.


In [356]:
# Load your Sentinel Hub credentials
config = SHConfig(
    sh_base_url="https://sh.dataspace.copernicus.eu",
    sh_token_url="https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
    sh_client_id=os.getenv("sentinel_hub_client_id"),
    sh_client_secret=os.getenv("sentinel_hub_client_secret"),
)

## Step 4: Load the GeoJSON Area of Interest (AOI)

Here, we load the AOI from a GeoJSON file and extract its boundaries.


In [357]:
# Load the GeoJSON area of interest (AOI)
aoi = gpd.read_file("extraction_areas/mumbai_offshore_subset.geojson")

# Convert the AOI to a bounding box (min_x, min_y, max_x, max_y format)
min_x, min_y, max_x, max_y = aoi.total_bounds
bbox = BBox(bbox=[min_x, min_y, max_x, max_y], crs=CRS.WGS84)  # type:ignore[arg-type]


## Step 5: Define Bounding Box Dimensions

In this step, we define the size of the bounding box area in pixels based on the desired resolution.


In [358]:
resolution = 0.75
# Define the size of the bounding box area (in pixels)
bbox_size = bbox_to_dimensions(bbox, resolution=resolution)  # Adjust resolution as needed
print(f"Image shape at {resolution} m resolution: {bbox_size} pixels")

Image shape at 0.75 m resolution: (2405, 1463) pixels


## Step 6: Create Sentinel Hub Requests for SAR Data

Here, we configure the request for Sentinel-1 SAR data with the specified evaluation script.

In [359]:
from datetime import datetime, timedelta

start_date = datetime(2018, 9, 1)
end_date = datetime(2019, 9, 30)
delta = timedelta(days=7)  # Change this to set your desired interval

current_date = start_date

In [360]:
all_requests = []

while current_date <= end_date:
    request = SentinelHubRequest(
        data_folder="./downloads",
        evalscript="""
        //VERSION=3
        function setup() {
          return {
            input: ["VV"],
            output: { bands: 1 }
          };
        }

        function evaluatePixel(sample) {
          return [sample.VV];
        }
        """,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL1_IW.define_from(
                    "s1iw", service_url=config.sh_base_url
                ),
                time_interval=(current_date.strftime('%Y-%m-%d'), current_date.strftime('%Y-%m-%d')),
            )
        ],
        bbox=bbox,
        size=bbox_size,
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        config=config,
    )
    all_requests.append((request, current_date))
    
    current_date += delta  # Move to the next date


In [375]:
data = SentinelHubDownloadClient(config=config).download([request.download_list[0] for (request, current_date) in all_requests], max_threads=5)

## Step 7: Visualize Downloaded Data

In [376]:
# Assuming data contains the images retrieved
num_images = len(data)  # Number of images from requests

# Define number of columns for the subplot
num_cols = 3  # Set the number of columns
num_rows = int(np.ceil(num_images / num_cols))  # Calculate number of rows

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, num_rows * 2), constrained_layout=True)  # Use constrained layout

# Flatten the axes array for easier indexing
axes = axes.flatten()

for i in range(num_images):
    axes[i].imshow(data[i], cmap='gray', vmin=0, vmax=255)
    axes[i].set_title(f'{all_requests[i][1]} | Week {i + 1}/{num_images}')  # Title for each subplot
    axes[i].axis('off')  # Turn off the axis

# Turn off any unused subplots
for j in range(num_images, len(axes)):
    axes[j].axis('off')  # Hide unused subplots

# Create a shared colorbar
# Adjusting the size and location of the colorbar
cbar = plt.colorbar(axes[0].images[0], ax=axes, orientation='vertical', fraction=0.02, pad=0.04, shrink=0.8)
cbar.set_label('Intensity')

# Set the main title
plt.suptitle('Sentinel-1 SAR Data Visualization (C-band, VV polarization)', fontsize=16)

# Use tight layout to ensure no overlap
plt.show()